In [ ]:
# ============================================================
# Libraries & settings
# ============================================================

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Reproducibility (random_state) and consistent plotting style across the notebook.

random_state = 42

sns.set_style("whitegrid")

# ============================================================
# Dataset loading (fail-fast: all-or-nothing)
# ============================================================

def upload_datasets(dict_paths):
  """
  Load all required CSV files into a dictionary of DataFrames.
  Fail-fast policy: if at least one file is missing, stop execution to avoid incomplete analyses.
  """
  dataframes={}
  missing = []

  for gas_name, paths in dict_paths.items():
      try:
          df=pd.read_csv(paths)
          print(f'{gas_name} upload successful. Rows: {df.shape[0]}')
          dataframes[gas_name]=df
      except FileNotFoundError:
          missing.append(paths)

  if missing:
        raise FileNotFoundError(f"Missing dataset files: {missing}")

  return dataframes

datasets = {
    'CH4' : 'CH4_Emissions.csv',
    'CO2' : 'CO2_Emissions.csv',
    'GHG' : 'GHG_Emissions.csv',
    'N2O' : 'N2O_Emissions.csv',
    'NOx' : 'NOx_Emissions.csv',
    'REP' : 'Renewable_Electricity_Production_Dataset.csv',
    'SO2' : 'SO2_emissions.csv'
}

df_emissions = upload_datasets(datasets)

In [ ]:
# ============================================================
# Dataset inspection
# ============================================================

for name, df in df_emissions.items():
    print(f"\n===== {name} =====")
    df.info()
    print(df.head())

In [ ]:
# ============================================================
# EU filtering (scope: EU-27)
# ============================================================

european_countries = [
    'Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia',
    'Denmark','Estonia','Finland','France','Germany','Greece',
    'Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg',
    'Malta','Netherlands','Poland','Portugal','Romania','Slovakia',
    'Slovenia','Spain','Sweden'
]

def eu_countries(df):
    return df[df['Country'].isin(european_countries)].reset_index(drop=True)

In [ ]:
filtered_gas = {
    gas: eu_countries(df)
    for gas, df in df_emissions.items()
    if gas != 'REP'
}

for gas, df in filtered_gas.items():
    print(f"\nNaN count for {gas}:")
    print(df.isna().sum())

filtered_ch4 = filtered_gas['CH4']
filtered_co2 = filtered_gas['CO2']
filtered_ghg = filtered_gas['GHG']
filtered_n2o = filtered_gas['N2O']
filtered_nox = filtered_gas['NOx']
filtered_so2 = filtered_gas['SO2']

In [ ]:
# ============================================================
# Emissions feature engineering (Total + Average)
# ============================================================

def convert_values_to_numeric(df):
  """
  Convert all non-'Country' columns to numeric.
  Strict mode (errors='raise'): if a value cannot be parsed, stop execution.
  """
  df = df.copy()
  numeric_cols = []
  for col in df.columns:
    if col != 'Country':
      df[col] = pd.to_numeric(df[col], errors='raise')
      numeric_cols.append(col)
  return df, numeric_cols

def calculate_emissions(df):
  """Compute total and average emissions per country across all year columns."""
  df = df.copy()
  df, numeric_cols = convert_values_to_numeric(df)
  df['Total Emissions'] = df[numeric_cols].sum(axis=1)
  df['Average Emissions'] = df[numeric_cols].mean(axis=1)
  return df

In [ ]:
def emissions_summary_plots(filtered_df, gas_name, figsize=(16, 6)):
    """
    Create two horizontal bar charts per gas:
    - Total emissions by country
    - Average emissions by country
    Both sorted descending for readability.
    """
    df_emissions = calculate_emissions(filtered_df)

    sorted_total = df_emissions.sort_values(by='Total Emissions', ascending=False)
    sorted_avg = df_emissions.sort_values(by='Average Emissions', ascending=False)

    fig, axes = plt.subplots(1, 2, figsize=figsize)

    axes[0].barh(sorted_total['Country'], sorted_total['Total Emissions'],color='blue')
    axes[0].set_xlabel('Total Emissions (Tons)')
    axes[0].set_ylabel('Country')
    axes[0].set_title(f'Total Emissions by Country ({gas_name})')
    axes[0].invert_yaxis()

    axes[1].barh(sorted_avg['Country'], sorted_avg['Average Emissions'],color='orange')
    axes[1].set_xlabel('Average Emissions (Tons)')
    axes[1].set_ylabel('Country')
    axes[1].set_title(f'Average Emissions by Country ({gas_name})')
    axes[1].invert_yaxis()

    plt.tight_layout()
    plt.show()

    return df_emissions, sorted_total, sorted_avg

ch4, sorted_total_ch4, sorted_average_ch4 = emissions_summary_plots(filtered_ch4, 'CH4')
co2, sorted_total_co2, sorted_average_co2 = emissions_summary_plots(filtered_co2, 'CO2')
ghg, sorted_total_ghg, sorted_average_ghg = emissions_summary_plots(filtered_ghg, 'GHG')
n2o, sorted_total_n2o, sorted_average_n2o = emissions_summary_plots(filtered_n2o, 'N2O')
nox, sorted_total_nox, sorted_average_nox = emissions_summary_plots(filtered_nox, 'NOx')
so2, sorted_total_so2, sorted_average_so2 = emissions_summary_plots(filtered_so2, 'SO2')

# **Goal 1**

In [ ]:
# ============================================================
# Identify the most polluting countries
# ============================================================

plt.figure(figsize=(14, 6))
sns.barplot(
    data=sorted_average_ghg.head(10),
    x='Average Emissions',
    y='Country'
)
for index, value in enumerate(sorted_average_ghg.head(10)['Average Emissions']):
    plt.text(
        value * 1.01,
        index,
        f'{value:,.0f}',
        va='center',
        ha='left'
    )
plt.xlim(0, sorted_average_ghg.head(10)['Average Emissions'].max() * 1.1)
plt.title('Top 10 EU Countries for GHG Emissions')
plt.xlabel('GHG Average Emissions')
plt.ylabel('Country')

plt.show()

In [ ]:
print(
    "Across all charts, Germany appears as the highest overall emitter, ranking first (or among the first) for CO2, CH4, GHG and NOx.\n"
    "France and Italy follow with consistently high values, especially for CO2, GHG and N2O.\n"
    "Poland stands out for SO2 and NOx, likely linked to a higher reliance on fossil fuels.\n"
    "Overall, larger and more industrialized economies contribute the most to EU emissions."
)

In [ ]:
def plot_trend_by_countries(df,gas_name):
  """Plot time-series emissions trends for each country (historical years only)."""
  plt.figure(figsize=(18, 8))
  years = df.columns[1:-2].astype(int)
  for country in df['Country'].unique():
    country_data = df[df['Country'] == country]
    plt.plot(years,country_data.iloc[0, 1:-2], label = country)
  plt.title(f'{gas_name} Emissions trends by Country', fontsize = 15)
  plt.xlabel('Years', fontsize = 12)
  plt.ylabel('Emissions (Tons)', fontsize = 12)
  plt.legend(title = 'Country', bbox_to_anchor = (1.05, 1), loc = 'upper left')
  plt.tight_layout()
  plt.show()

gas_dfs = {
    'CH4': ch4,
    'CO2': co2,
    'GHG': ghg,
    'N2O': n2o,
    'NOx': nox,
    'SO2': so2
}

for gas, df in gas_dfs.items():
    plot_trend_by_countries(df, gas)

# **Goal 2**

In [ ]:
# ============================================================
# Goal 2 — EU-wide GHG average trend over time
# ============================================================

plot_mean_years = ghg.columns[1:-2]   # Year columns only (exclude Country + engineered features)

ue_mean_emissions = ghg[plot_mean_years].mean(axis=0)

ue_trend = pd.DataFrame({
    'Year': plot_mean_years.astype(int),
    'Average_Emissions_UE': ue_mean_emissions.values
})

plt.figure(figsize=(16, 5))
sns.lineplot(data=ue_trend, x='Year', y='Average_Emissions_UE',marker='o')
plt.title('EU-wide trend in average GHG emissions')
plt.xlabel('Years')
plt.ylabel('Average Emissions(Tons)')
plt.show()


In [ ]:
print(
    "EU-wide GHG emissions show a broadly declining trend, especially after 2005.\n"
    "Germany, France and Italy display consistent reductions (with a visible drop around 2019–2020).\n"
    "A partial rebound appears in 2021–2022, likely related to post-pandemic economic recovery.\n"
    "Some Eastern countries (e.g., Poland) show slower reductions for NOx and SO2.\n"
    "Overall, the long-term trend remains positive, suggesting improving environmental policies."
)

In [ ]:
# ============================================================
# REP dataset preparation (different schema)
# ============================================================

df_rep = df_emissions['REP'].copy()

df_rep.info()

df_rep.head()

In [ ]:
df_rep.rename(columns={'Country and area':'Country'}, inplace=True)

df_rep.head()

In [ ]:
filtered_rep = eu_countries(df_rep)

filtered_rep.info()

In [ ]:
# Assumption (project constraint): the REP dataset schema is stable, year columns are strings, ordered, and always include "2005".

def from_2005_to_now(df):
  """Return all columns from 2005 onward (REP dataset is wide: one column per year)."""
  return list(df.loc[:, '2005':])

rep_data_2005 = from_2005_to_now(filtered_rep)

rep_clean = filtered_rep [['Country']+ rep_data_2005]

rep_clean.head()

In [ ]:
rep_clean_copy = rep_clean.copy()

best_2023_rep = rep_clean_copy[['Country','2023']].sort_values(by='2023', ascending=False)

best_2023_rep

In [ ]:
plt.figure(figsize=(15, 6))
plt.barh(best_2023_rep['Country'], best_2023_rep['2023'], color = 'green')
plt.xlabel('Renewable Electricity Production (%)')
plt.ylabel('Country')
plt.title('Top Countries with Highest Renewable Electricity Production in 2023')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Country as Index

heatmap_data = rep_clean_copy.set_index('Country')

plt.figure(figsize=(16, 6))
sns.heatmap(
    heatmap_data,
    cmap='YlGn',
    linewidths=0.2,
    cbar_kws={'label': 'Percentage'}
)

plt.title(f'Trend Renewable Electricity Production (%)', fontsize = 16)
plt.xlabel('Years', fontsize = 12)
plt.ylabel('Countries', fontsize = 12)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Goal 3 — Correlation: REP vs emissions (mean and total)
# ============================================================
# Precompute REP mean/total once, then merge per gas on Country for correlation.

def check_correlation(df, gas_name):
  """Compute Pearson correlation between REP mean (%) and gas average emissions."""
  filtered_gas = df.copy()
  # Project constraint: all gas datasets include year columns and always include "2005"
  columns_2005 = [col for col in from_2005_to_now(filtered_gas) if col.isdigit()]
  filtered_gas_2005 = filtered_gas[['Country'] + columns_2005].copy()
  filtered_gas_2005['Average Emissions'] = filtered_gas_2005.iloc[:, 1:].mean(axis=1)
  df_gas_mean = filtered_gas_2005[['Country', 'Average Emissions']]
  df_correlation = pd.merge(df_rep_mean, df_gas_mean, on='Country')
  correlation_value = df_correlation['Renewable_Energy_Mean'].corr(df_correlation['Average Emissions'])

  print(f'Correlation between renewable production and {gas_name}: {correlation_value:.2f}')
  return df_correlation, correlation_value

rep_mean_series = rep_clean[rep_data_2005].mean(axis=1)

df_rep_mean = pd.DataFrame({
    'Country': rep_clean['Country'].values,
    'Renewable_Energy_Mean': rep_mean_series.values
})

ch4_correlation, corr_ch4 = check_correlation(filtered_ch4, 'CH4')

ghg_correlation, corr_ghg = check_correlation(filtered_ghg, 'GHG')

co2_correlation, corr_co2 = check_correlation(filtered_co2, 'CO2')

n2o_correlation, corr_n2o = check_correlation(filtered_n2o, 'N2O')

nox_correlation, corr_nox = check_correlation(filtered_nox, 'NOx')

so2_correlation, corr_so2 = check_correlation(filtered_so2, 'SO2')

print(
    "All correlations are negative: higher renewable electricity production tends to be associated with lower emissions.\n"
    "However, the relationship is weak-to-moderate (roughly between -0.18 and -0.30), so renewables are not the only driver.\n"
    "The strongest pattern appears for SO2, which is closely tied to fossil fuel combustion.\n"
    "For CH4, CO2, N2O and NOx the link is weaker, likely because large portions come from non-electric sectors (e.g., transport, industry, agriculture)."
)

In [ ]:
def check_correlation_total(df, gas_name):
  """Compute Pearson correlation between REP total and gas total emissions."""
  filtered_gas_total = df.copy()
  # Project constraint: all gas datasets include year columns and always include "2005"
  columns_2005_total = [col for col in from_2005_to_now(filtered_gas_total) if col.isdigit()]
  filtered_gas_2005_total = filtered_gas_total[['Country'] + columns_2005_total].copy()
  filtered_gas_2005_total['Total Emissions'] = filtered_gas_2005_total.iloc[:, 1:].sum(axis=1)
  df_gas_total = filtered_gas_2005_total[['Country', 'Total Emissions']]
  df_correlation_total = pd.merge(df_rep_total, df_gas_total, on='Country')
  correlation_value_total = df_correlation_total['Renewable_Energy_Total'].corr(df_correlation_total['Total Emissions'])

  print(f'Correlation between renewable production and {gas_name}: {correlation_value_total:.2f}')
  return df_correlation_total, correlation_value_total

df_rep_total = rep_clean[rep_data_2005].sum(axis=1)

df_rep_total = pd.DataFrame({
    'Country': rep_clean['Country'].values,
    'Renewable_Energy_Total': df_rep_total.values
})

ch4_correlation_total, corr_ch4_total = check_correlation_total(filtered_ch4, 'CH4')

ghg_correlation_total, corr_ghg_total = check_correlation_total(filtered_ghg, 'GHG')

co2_correlation_total, corr_co2_total = check_correlation_total(filtered_co2, 'CO2')

n2o_correlation_total, corr_n2o_total = check_correlation_total(filtered_n2o, 'N2O')

nox_correlation_total, corr_nox_total = check_correlation_total(filtered_nox, 'NOx')

so2_correlation_total, corr_so2_total = check_correlation_total(filtered_so2, 'SO2')

print(
    "Mean and total correlations match here because totals are a constant scaling of means (same time window),\n"
    "and Pearson correlation is invariant to positive linear scaling."
)

In [ ]:
# Scatter plots (REP mean vs total emissions)

# CH4
plt.figure(figsize=(12, 6))
plt.scatter(
    ch4_correlation['Renewable_Energy_Mean'],
    ch4_correlation_total['Total Emissions'],
    label = f'CH4 (Corr:{corr_ch4_total:.2f})',
    color = 'blue',
    alpha = 0.7
)

# N2O

plt.scatter(
    n2o_correlation['Renewable_Energy_Mean'],
    n2o_correlation_total['Total Emissions'],
    label = f'N2O (Corr:{corr_n2o_total:.2f})',
    color = 'purple',
    alpha = 0.7
)

# NOx

plt.scatter(
    nox_correlation['Renewable_Energy_Mean'],
    nox_correlation_total['Total Emissions'],
    label = f'NOx (Corr:{corr_nox_total:.2f})',
    color = 'orange',
    alpha = 0.7
)

# SO2

plt.scatter(
    so2_correlation['Renewable_Energy_Mean'],
    so2_correlation_total['Total Emissions'],
    label = f'SO2 (Corr:{corr_so2_total:.2f})',
    color = 'red',
    alpha = 0.7
)

plt.title('Correlation Between Renewable Energy & Gas Pollution', fontsize = 16)
plt.xlabel('Average Renewable Energy Production (%)', fontsize = 12)
plt.ylabel('Total Emissions (Tons)', fontsize = 12)
plt.legend(title='Gas Type', fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# CO2
plt.figure(figsize=(12, 6))
plt.scatter(
    co2_correlation['Renewable_Energy_Mean'],
    co2_correlation_total['Total Emissions'],
    label = f'CO2 (Corr:{corr_co2_total:.2f})',
    color = 'indigo',
    alpha = 0.7
)

plt.title('Correlation Between Renewable Energy & CO2', fontsize = 16)
plt.xlabel('Average Renewable Energy Production (%)', fontsize = 12)
plt.ylabel('Total Emissions (Tons)', fontsize = 12)
plt.legend(title='Gas Type', fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# **Goal 3**

In [ ]:
# GHG
plt.figure(figsize=(12, 6))
plt.scatter(
    ghg_correlation['Renewable_Energy_Mean'],
    ghg_correlation_total['Total Emissions'],
    label = f'GHG (Corr:{corr_ghg_total:.2f})',
    color = 'darkblue',
    alpha = 0.7
)

plt.title('Correlation Between Renewable Energy & GHG', fontsize = 16)
plt.xlabel('Average Renewable Energy Production (%)', fontsize = 12)
plt.ylabel('Total Emissions (Tons)', fontsize = 12)
plt.legend(title='Gas Type', fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
print(
    "Scatter plots confirm a negative but weak association between renewable production and emissions.\n"
    "The dispersion is high, suggesting that additional structural factors influence national emissions beyond electricity generation."
)

In [ ]:
# -------------------------
# Forecasting CO2 (2024–2035) - Linear Regression
# -------------------------

co2 = co2.copy()

years = np.array([int(year) for year in co2.columns[1:-2]]).reshape(-1, 1)

future_years = np.array(range(2024,2036)).reshape(-1,1)

predictions = pd.DataFrame({'Year': range(2024, 2036)})

metrics_df = pd.DataFrame(columns=['Country','MAE','RMSE','R2'])

for country in co2['Country'].unique():
  country_data_predictions = co2[co2['Country']==country].iloc[:,1:-2].values.flatten()
  model = LinearRegression()
  model.fit(years, country_data_predictions)
  y_pred = model.predict(years)
  mae = mean_absolute_error(country_data_predictions,y_pred)
  mse = mean_squared_error(country_data_predictions,y_pred)
  rmse = np.sqrt(mse)
  r2 = r2_score(country_data_predictions,y_pred)
  metrics_df.loc[len(metrics_df)] = {
      'Country': country,
      'MAE': mae,
      'RMSE': rmse,
      'R2': r2
  }

  future_predictions = model.predict(future_years)

  predictions[country] = future_predictions

plt.figure(figsize=(16,6))

co2_countries = co2['Country'].unique()
c_palette = plt.cm.tab20.colors
c_map = {country: c_palette[i % len(c_palette)] for i, country in enumerate(co2_countries)}

hist_years = co2.columns[1:-2].astype(int)
fut_years = predictions['Year']

for country in co2_countries:
  plt.plot(
      hist_years,
      co2[co2['Country']==country].iloc[:,1:-2].values.flatten(),
      label = f'{country} (historical)',
      color = c_map[country]
  )

for country in predictions.columns[1:]:
  plt.plot(
      fut_years,
      predictions[country],
      linestyle = '--',
      label = f'{country} (future)',
      color = c_map[country]
  )

plt.title('Prediction of CO2 Emissions for EU Countries')
plt.xlabel('Year')
plt.ylabel('CO2 Emissions (Tons)')
plt.legend(ncol=2, loc='upper left', bbox_to_anchor=(1, 1), fontsize = 10)
plt.tight_layout()
plt.show()

In [ ]:
print(metrics_df.sort_values("R2", ascending=False).round(3))

print(
    "Results vary widely across countries: some trends are well captured, while others show weaker fit.\n"
    "This is expected when using a simple baseline model across heterogeneous national profiles."
)

In [ ]:
# -------------------------
# Forecasting CO2 (2024–2035) - Random Forest Regressor
# -------------------------

years_rfr = np.array([int(year) for year in co2.columns[1:-2]]).reshape(-1, 1)

future_years_rfr = np.array(range(2024,2036)).reshape(-1,1)

predictions_rfr = pd.DataFrame({'Year': range(2024, 2036)})

metrics_df_rfr = pd.DataFrame(columns=['Country','MAE','RMSE','R2'])

for country in co2['Country'].unique():
  country_data_predictions_rfr = co2[co2['Country']==country].iloc[:,1:-2].values.flatten()
  model_rfr = RandomForestRegressor(n_estimators=100,random_state=random_state)
  model_rfr.fit(years_rfr, country_data_predictions_rfr)
  y_pred_rfr = model_rfr.predict(years_rfr)
  mae_rfr = mean_absolute_error(country_data_predictions_rfr,y_pred_rfr)
  mse_rfr = mean_squared_error(country_data_predictions_rfr,y_pred_rfr)
  rmse_rfr = np.sqrt(mse_rfr)
  r2_rfr = r2_score(country_data_predictions_rfr,y_pred_rfr)
  metrics_df_rfr.loc[len(metrics_df_rfr)] = {
      'Country': country,
      'MAE': mae_rfr,
      'RMSE': rmse_rfr,
      'R2': r2_rfr
  }

  future_predictions_rfr = model_rfr.predict(future_years_rfr)

  predictions_rfr[country] = future_predictions_rfr

plt.figure(figsize=(16,6))

countries_rfr = co2['Country'].unique()
c_palette_rfr = plt.cm.tab20.colors
c_map_rfr = {country: c_palette_rfr[i % len(c_palette_rfr)] for i, country in enumerate(countries_rfr)}

hist_years_rfr = co2.columns[1:-2].astype(int)
fut_years_rfr = predictions_rfr['Year']

for country in countries_rfr:
  plt.plot(
      hist_years_rfr,
      co2[co2['Country']==country].iloc[:,1:-2].values.flatten(),
      label  = f'{country} (historical)',
      color = c_map_rfr[country]
  )

for country in predictions_rfr.columns[1:]:
  plt.plot(
      fut_years_rfr,
      predictions_rfr[country],
      linestyle = '--',
      label = f'{country} (future)',
      color = c_map_rfr[country]
  )

plt.title('Prediction of CO2 Emissions for EU Countries')
plt.xlabel('Year')
plt.ylabel('CO2 Emissions (Tons)')
plt.legend(ncol=2, loc='upper left', bbox_to_anchor=(1, 1), fontsize = 10)
plt.tight_layout()
plt.show()

In [ ]:
print(metrics_df_rfr.sort_values("R2", ascending=False).round(3))

print(
    "Random Forest performs extremely well in-sample, but it is not designed for reliable long-horizon extrapolation.\n"
    "Here it is used only as an exploratory benchmark."
)

In [ ]:
# -------------------------
# Forecasting CO2 (2024–2035) - Linear Regression Polynomial
# -------------------------


years_lrp = np.array([int(year) for year in co2.columns[1:-2]]).reshape(-1, 1)

future_years_lrp = np.array(range(2024,2036)).reshape(-1,1)

predictions_lrp = pd.DataFrame({'Year': range(2024, 2036)})

degree = 2

metrics_df_lrp = pd.DataFrame(columns=['Country','MAE','RMSE','R2'])

for country in co2['Country'].unique():
  country_data_predictions_lrp = co2[co2['Country']==country].iloc[:,1:-2].values.flatten()
  poly = PolynomialFeatures(degree=degree)
  years_poly = poly.fit_transform(years_lrp)
  model_lrp = LinearRegression()
  model_lrp.fit(years_poly, country_data_predictions_lrp)
  y_pred_lrp = model_lrp.predict(years_poly)
  mae_lrp = mean_absolute_error(country_data_predictions_lrp,y_pred_lrp)
  mse_lrp = mean_squared_error(country_data_predictions_lrp,y_pred_lrp)
  rmse_lrp = np.sqrt(mse_lrp)
  r2_lrp = r2_score(country_data_predictions_lrp,y_pred_lrp)
  metrics_df_lrp.loc[len(metrics_df_lrp)] = {
      'Country': country,
      'MAE': mae_lrp,
      'RMSE': rmse_lrp,
      'R2': r2_lrp
  }

  future_years_poly = poly.transform(future_years_lrp.reshape(-1,1))

  future_predictions_lrp = model_lrp.predict(future_years_poly)

  # Emissions cannot be negative

  future_predictions_lrp = np.maximum(future_predictions_lrp, 0)

  predictions_lrp[country] = future_predictions_lrp

plt.figure(figsize=(16,6))

countries_lrp = co2['Country'].unique()
c_palette_lrp = plt.cm.tab20.colors
c_map_lrp = {country: c_palette_lrp[i % len(c_palette_lrp)] for i, country in enumerate(countries_lrp)}

hist_years_lrp = co2.columns[1:-2].astype(int)
fut_years_lrp = predictions_lrp['Year']

for country in countries_lrp:
  plt.plot(
      hist_years_lrp,
      co2[co2['Country']==country].iloc[:,1:-2].values.flatten(),
      label  = f'{country} (historical)',
      color = c_map_lrp[country]
  )

for country in predictions_lrp.columns[1:]:
  plt.plot(
      fut_years_lrp,
      predictions_lrp[country],
      linestyle = '--',
      label = f'{country} (future)',
      color = c_map_lrp[country]
  )

plt.title('Prediction of CO2 Emissions for EU Countries')
plt.xlabel('Year')
plt.ylabel('CO2 Emissions (Tons)')
plt.legend(ncol=2, loc='upper left', bbox_to_anchor=(1, 1), fontsize = 10)
plt.tight_layout()
plt.show()

# **Goal 4**

In [ ]:
# Heatmap (historical + forecast)

plt.figure(figsize=(18, 7))

sns.heatmap(
    pd.concat(
        [
            co2.set_index('Country').iloc[:,1:-2].rename(columns=lambda x: int(x)),
            predictions_lrp.set_index('Year').T.reindex(co2['Country']).rename(columns=lambda x: int(x))
        ],
        axis=1
    ),
    cmap='Blues',
    vmin=0,
    cbar_kws={'label': 'CO2 Emissions (Tons)'}
)

plt.title('Prediction of CO2 Emissions for EU Countries')
plt.xlabel('Year')
plt.ylabel('Country')

plt.axvline(len(hist_years_lrp), color='black', linewidth=2)

plt.tight_layout()
plt.show()


In [ ]:
print(metrics_df_lrp.sort_values("R2", ascending=False).round(3))

print(
    "The degree-2 polynomial model does not provide a consistent improvement over the linear baseline,\n"
    "suggesting that CO2 trends in this period are largely linear."
)

In [ ]:
print(
    "To forecast EU CO2 emissions, I compared three regression approaches: Linear Regression, "
    "2nd-degree Polynomial Regression, and a Random Forest Regressor.\n"
    "Linear Regression works well as a strong baseline: it captures the overall direction of the trend and "
    "provides the most coherent long-horizon extrapolation, even if performance varies across countries.\n"
    "The 2nd-degree polynomial model does not deliver a consistent improvement over the linear baseline, "
    "suggesting that CO2 dynamics in this time window are largely linear.\n"
    "Random Forest achieves excellent in-sample metrics (high R²), but this is mainly due to overfitting and "
    "it is not reliable for time extrapolation, leading to less realistic long-term forecasts.\n"
    "Overall, Linear Regression is the most appropriate choice here thanks to its interpretability, "
    "stable future trajectories, and suitability for policy-oriented analysis."
)

In [ ]:
# -------------------------
# Forecasting REP (2024–2035) - Linear Regression
# -------------------------

years_rep = np.array([int(year) for year in rep_clean_copy.columns[1:]]).reshape(-1, 1)

future_years_rep = np.array(range(2024,2036)).reshape(-1,1)

predictions_rep = pd.DataFrame({'Year': range(2024, 2036)})

metrics_df_rep = pd.DataFrame(columns=['Country','MAE','RMSE','R2'])

for country in rep_clean_copy['Country'].unique():
  country_data_predictions_rep = rep_clean_copy[rep_clean_copy['Country']==country].iloc[:,1:].values.flatten()
  model_rep = LinearRegression()
  model_rep.fit(years_rep, country_data_predictions_rep)
  y_pred_rep = model_rep.predict(years_rep)
  mae_rep = mean_absolute_error(country_data_predictions_rep,y_pred_rep)
  mse_rep = mean_squared_error(country_data_predictions_rep,y_pred_rep)
  rmse_rep = np.sqrt(mse_rep)
  r2_rep = r2_score(country_data_predictions_rep,y_pred_rep)
  metrics_df_rep.loc[len(metrics_df_rep)] = {
      'Country': country,
      'MAE': mae_rep,
      'RMSE': rmse_rep,
      'R2': r2_rep
  }

  # REP is a percentage: enforce plausible bounds [0, 100]

  future_predictions_rep = model_rep.predict(future_years_rep)

  future_predictions_rep = np.clip(future_predictions_rep, 0, 100)

  predictions_rep[country] = future_predictions_rep

plt.figure(figsize=(16,6))

rep_countries = rep_clean_copy['Country'].unique()
c_palette = plt.cm.tab20.colors
c_map = {country: c_palette[i % len(c_palette)] for i, country in enumerate(rep_countries)}

hist_years_rep = rep_clean_copy.columns[1:].astype(int)
fut_years_rep = predictions_rep['Year']

for country in rep_countries:
  plt.plot(
      hist_years_rep,
      rep_clean_copy[rep_clean_copy['Country']==country].iloc[:,1:].values.flatten(),
      label = f'{country} (historical)',
      color = c_map[country]
  )

for country in predictions_rep.columns[1:]:
  plt.plot(
      fut_years_rep,
      predictions_rep[country],
      linestyle = '--',
      label = f'{country} (future)',
      color = c_map[country]
  )

plt.title('Prediction of Renewable Energy Production for EU Countries')
plt.xlabel('Year')
plt.ylabel('R.E.P. Production (%)')
plt.legend(ncol=2, loc='upper left', bbox_to_anchor=(1, 1), fontsize = 10)
plt.tight_layout()
plt.show()

In [ ]:
print(metrics_df_rep.sort_values("R2", ascending=False).round(3))

print(
    "Linear regression captures growth trends reasonably well for many countries.\n"
    "In others, performance is weaker due to higher volatility or non-linear dynamics.\n"
    "Overall, it provides a clear and interpretable baseline."
)

In [ ]:
# -------------------------
# Forecasting REP (2024–2035) -  Random Forest Regressor
# -------------------------

years_rfr_rep = np.array([int(year) for year in rep_clean_copy.columns[1:]]).reshape(-1, 1)

future_years_rfr_rep = np.array(range(2024,2036)).reshape(-1,1)

predictions_rfr_rep = pd.DataFrame({'Year': range(2024, 2036)})

metrics_df_rfr_rep = pd.DataFrame(columns=['Country','MAE','RMSE','R2'])

for country in rep_clean_copy['Country'].unique():
  country_data_predictions_rfr_rep = rep_clean_copy[rep_clean_copy['Country']==country].iloc[:,1:].values.flatten()
  model_rfr_rep = RandomForestRegressor(n_estimators=100,random_state=random_state)
  model_rfr_rep.fit(years_rfr_rep, country_data_predictions_rfr_rep)
  y_pred_rfr_rep = model_rfr_rep.predict(years_rfr_rep)
  mae_rfr_rep = mean_absolute_error(country_data_predictions_rfr_rep,y_pred_rfr_rep)
  mse_rfr_rep= mean_squared_error(country_data_predictions_rfr_rep,y_pred_rfr_rep)
  rmse_rfr_rep = np.sqrt(mse_rfr_rep)
  r2_rfr_rep = r2_score(country_data_predictions_rfr_rep,y_pred_rfr_rep)
  metrics_df_rfr_rep.loc[len(metrics_df_rfr_rep)] = {
      'Country': country,
      'MAE': mae_rfr_rep,
      'RMSE': rmse_rfr_rep,
      'R2': r2_rfr_rep
  }

  future_predictions_rfr_rep = model_rfr_rep.predict(future_years_rfr_rep)

  predictions_rfr_rep[country] = future_predictions_rfr_rep

plt.figure(figsize=(16,6))

countries_rfr_rep = rep_clean_copy['Country'].unique()
c_palette_rfr_rep = plt.cm.tab20.colors
c_map_rfr_rep = {country: c_palette_rfr_rep[i % len(c_palette_rfr_rep)] for i, country in enumerate(countries_rfr_rep)}

hist_years_rfr_rep = rep_clean_copy.columns[1:].astype(int)
fut_years_rfr_rep = predictions_rfr_rep['Year']

for country in countries_rfr_rep:
  plt.plot(
      hist_years_rfr_rep,
      rep_clean_copy[rep_clean_copy['Country']==country].iloc[:,1:].values.flatten(),
      label  = f'{country} (historical)',
      color = c_map_rfr_rep[country]
  )

for country in predictions_rfr_rep.columns[1:]:
  plt.plot(
      fut_years_rfr_rep,
      predictions_rfr_rep[country],
      linestyle = '--',
      label = f'{country} (future)',
      color = c_map_rfr_rep[country]
  )

plt.title('Prediction of Renewable Energy Production for EU Countries')
plt.xlabel('Year')
plt.ylabel('R.E.P. Production (%)')
plt.legend(ncol=2, loc='upper left', bbox_to_anchor=(1, 1), fontsize = 10)
plt.tight_layout()
plt.show()

In [ ]:
print(metrics_df_rfr_rep.sort_values("R2", ascending=False).round(3))

print(
    "Random Forest fits historical REP extremely well, but tends to produce flat/average-like future paths.\n"
    "This limits its usefulness for long-term forecasting and makes it mainly exploratory here."
)

In [ ]:
# -------------------------
# Forecasting REP (2024–2035) -  Linear Regression Polynomial
# -------------------------

years_lrp_rep = np.array([int(year) for year in rep_clean_copy.columns[1:]]).reshape(-1, 1)

future_years_lrp_rep = np.array(range(2024,2036)).reshape(-1,1)

predictions_lrp_rep = pd.DataFrame({'Year': range(2024, 2036)})

degree_rep = 2

metrics_df_lrp_rep = pd.DataFrame(columns=['Country','MAE','RMSE','R2'])

for country in rep_clean_copy['Country'].unique():
  country_data_predictions_lrp_rep = rep_clean_copy[rep_clean_copy['Country']==country].iloc[:,1:].values.flatten()
  poly_rep = PolynomialFeatures(degree=degree_rep)
  years_poly_rep = poly_rep.fit_transform(years_lrp_rep)
  model_lrp_rep = LinearRegression()
  model_lrp_rep.fit(years_poly_rep, country_data_predictions_lrp_rep)
  y_pred_lrp_rep = model_lrp_rep.predict(years_poly_rep)
  mae_lrp_rep = mean_absolute_error(country_data_predictions_lrp_rep,y_pred_lrp_rep)
  mse_lrp_rep = mean_squared_error(country_data_predictions_lrp_rep,y_pred_lrp_rep)
  rmse_lrp_rep = np.sqrt(mse_lrp_rep)
  r2_lrp_rep = r2_score(country_data_predictions_lrp_rep,y_pred_lrp_rep)
  metrics_df_lrp_rep.loc[len(metrics_df_lrp_rep)] = {
      'Country': country,
      'MAE': mae_lrp_rep,
      'RMSE': rmse_lrp_rep,
      'R2': r2_lrp_rep
  }

  future_years_poly_rep = poly_rep.transform(future_years_lrp_rep.reshape(-1,1))

  future_predictions_lrp_rep = model_lrp_rep.predict(future_years_poly_rep)

  # REP is a percentage: enforce plausible bounds [0, 100]

  future_predictions_lrp_rep = np.clip(future_predictions_lrp_rep, 0, 100)

  predictions_lrp_rep[country] = future_predictions_lrp_rep

plt.figure(figsize=(16,6))

countries_lrp_rep = rep_clean_copy['Country'].unique()
c_palette_lrp = plt.cm.tab20.colors
c_map_lrp = {country: c_palette_lrp[i % len(c_palette_lrp)] for i, country in enumerate(countries_lrp_rep)}

hist_years_lrp_rep = rep_clean_copy.columns[1:].astype(int)
fut_years_lrp_rep = predictions_lrp_rep['Year']

for country in countries_lrp_rep:
  plt.plot(
      hist_years_lrp_rep,
      rep_clean_copy[rep_clean_copy['Country']==country].iloc[:,1:].values.flatten(),
      label  = f'{country} (historical)',
      color = c_map_lrp[country]
  )

for country in predictions_lrp_rep.columns[1:]:
  plt.plot(
      fut_years_lrp_rep,
      predictions_lrp_rep[country],
      linestyle = '--',
      label = f'{country} (future)',
      color = c_map_lrp[country]
  )

plt.title('Prediction of Renewable Energy Production for EU Countries')
plt.xlabel('Year')
plt.ylabel('R.E.P. Production (%)')
plt.legend(ncol=2, loc='upper left', bbox_to_anchor=(1, 1), fontsize = 10)
plt.tight_layout()
plt.show()

# **Goal 5**

In [ ]:
# Heatmap (historical + forecast)

plt.figure(figsize=(18, 7))

sns.heatmap(
    pd.concat(
        [
            rep_clean_copy.set_index('Country').iloc[:,1:].rename(columns=lambda x: int(x)),
            predictions_lrp_rep.set_index('Year').T.reindex(rep_clean_copy['Country']).rename(columns=lambda x: int(x))
        ],
        axis=1
    ),
    cmap='Greens',
    vmin=0,
    vmax=100,
    cbar_kws={'label': 'R.E.P. (%)'}
)

plt.title('Prediction of R.E.P. (%) for EU Countries')
plt.xlabel('Year')
plt.ylabel('Country')
plt.axvline(len(hist_years_lrp_rep), color='black', linewidth=2)

plt.tight_layout()
plt.show()

In [ ]:
print(metrics_df_lrp_rep.sort_values("R2", ascending=False).round(3))

print(
    "Polynomial regression can capture some non-linear patterns, improving fit for certain countries.\n"
    "However, it may also introduce instability in the forecasts, so the linear model remains the most robust baseline."
)